# Explorer les données d'Open Food Facts

Nous allons explorer le jeu de données d'Open Food Facts, filtré dans le notebook précédent pour ne conserver que

* les fiches de produits vendus en France,
* les champs utilisables,
* les fiches contenant un nom de produit et une marque,
* les fiches marquées "complètes".

L'application successive de ces filtres nous permet d'explorer un sous-ensemble du jeu de données de départ:
XXXX fiches avec XXX champs sélectionnées parmi X.X millions de fiches avec XXX champs.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
# modifier le chemin vers le fichier CSV si nécessaire
# ici on utilise le fichier filtré dans le notebook précédent
CSV_FILE = '../data/off_subset.csv'

In [4]:
# on charge les types de données qu'on a spécifiés dans le notebook 1
DTYPE_FILE = '../data/dtype.txt'
with open(DTYPE_FILE) as f:
    dtype = eval(f.read())

In [5]:
# TODO charger les données
df = pd.read_csv(CSV_FILE, sep='\t', encoding='utf-8')

/home/mathieu/miniconda3/envs/off-explore/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Analyse exploratoire par datavisualisation

L'analyse de données exploratoire repose crucialement sur la visualisation des données.
Les datavisualisations permettent d'embrasser le contenu d'un jeu de données par facettes, de mettre en évidence des tendances, des régularités, des corrélations...

Votre tâche est maintenant de produire un ensemble de visualisations de notre sous-ensemble d'Open Food Facts.
Vous devez produire des visualisations qui vous permettent de former des hypothèses, de mettre en évidence des phénomènes, de confirmer des intuitions...

Les visualisations peuvent porter sur des variables considérées isolément ou sur des paires (ou des triplets) de variables.
La quantité de données rend particulièrement intéressantes les visualisations produites en parallèle sur plusieurs groupes ou familles de produits.

Vous pouvez utiliser matplotlib et seaborn, en vous inspirant de la [gallerie d'exemples de seaborn](https://seaborn.pydata.org/examples/index.html), du tutorial par exemple sur les [visualisations par facettes](https://seaborn.pydata.org/tutorial/relational.html#showing-multiple-relationships-with-facets) ou d'un exemple de [visualisation 3d avec matplotlib](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html).

In [30]:
# TODO

Vous pouvez même générer des visualisations 3d interactives avec Jupyterlab et matplotlib si vous installez l'extension [jupyter-matplotlib](https://github.com/matplotlib/ipympl) (cf. https://stackoverflow.com/a/56416229/14201886 ).

In [31]:
# TODO (optionnel)

## Bonus : Données géographiques

Le champ `first_packaging_code_geo` d'OpenFoodFacts contient des coordonnées géographiques.

Il est donc possible de dessiner une carte avec `geopandas`:
https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html

In [32]:
# TODO dessiner une carte à partir de first_packaging_code_geo
# df_off[["fp_lat","fp_lon"]] = df_off["first_packaging_code_geo"].str.split(",", 1, expand=True)
# df_off.fp_lat = round(df_off.fp_lat.astype(float), 2)
# df_off.fp_lon = round(df_off.fp_lon.astype(float), 2)
# df_off = df_off.drop(columns=["first_packaging_code_geo"])
# TODO continue...